In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

# Model Definition with CNN layers
class SimpleCNN(nn.Module):
    def __init__(self, output_dim):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 7 * 7, 512)  # After 3 Conv layers, the image size is reduced to 7x7
        self.fc2 = nn.Linear(512, output_dim)
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer to downsample the image

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 7 * 7)  # Flatten the output of the last convolutional layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize Model, Loss Function, and Optimizer
model = SimpleCNN(output_dim=10)  # 10 classes for MNIST
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Accuracy Calculation
def accuracy_score(y_pred, y_true):
    _, predicted = torch.max(y_pred, 1)
    eq = torch.eq(predicted, y_true).sum()
    return (eq / len(y_true)) * 100

epochs = 10
print("[+] Training Start")

for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    train_accuracy = 0
    test_accuracy = 0
    total_train_samples = 0
    total_correct_train = 0

    # Training Phase
    model.train()
    for data, target in train_loader:
        y_pred = model(data)
        loss = loss_fn(y_pred, target)
        train_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(y_pred, 1)
        total_train_samples += target.size(0)
        total_correct_train += (predicted == target).sum().item()

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

    # Calculate training accuracy
    train_accuracy = total_correct_train / total_train_samples * 100

    # Testing Phase
    model.eval()
    with torch.inference_mode():
        for data, target in test_loader:
            y_pred = model(data)
            loss = loss_fn(y_pred, target)
            test_loss += loss.item()

            # Calculate test accuracy
            _, predicted = torch.max(y_pred, 1)
            test_accuracy += (predicted == target).sum().item()

    # Calculate test accuracy
    total_test_samples = len(test_loader.dataset)
    test_accuracy = test_accuracy / total_test_samples * 100

    # Print the results for this epoch
    avg_train_loss = train_loss / len(train_loader)
    avg_test_loss = test_loss / len(test_loader)

    print(f"{epoch+1}/{epochs} Train Loss: {avg_train_loss:.4f} Train Acc: {train_accuracy:.2f}%")
    print(f"{epoch+1}/{epochs} Test Loss: {avg_test_loss:.4f} Test Acc: {test_accuracy:.2f}%")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:05<00:00, 1.95MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 42.5kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:02<00:00, 795kB/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

[+] Training Start


RuntimeError: shape '[-1, 6272]' is invalid for input of size 36864